In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
import requests
import openpyxl
from openpyxl import Workbook
from bs4 import BeautifulSoup
from openpyxl.utils.dataframe import dataframe_to_rows
from tensorflow.keras import layers
from nltk.corpus import stopwords
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

# 환율 계산 셀(exchange) 

In [ ]:
exchange = 

# 재무제표 계산 셀(financial) 

# 유망주(삼성) 크롤링 

In [ ]:
target = '삼성전자'
wb = Workbook()
sheet = wb.active
sheet.append(['발행일','언론사','기사제목','URL','내용'])

headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

In [ ]:
response = requests.get(f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={target}', headers=headers)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
samsung_news = soup.select('div.news_wrap.api_ani_send')

In [ ]:
samsung_news

In [ ]:
# 필요없는 부분 다 떼고 본문만 남기기

# 주가 예측 

In [ ]:
# 전처리
DATA_IN_PATH = './data_in/'

train_data = pd.read_csv(DATA_IN_PATH + 'train.txt', header=0, delimiter='\t', quoting=3)

In [ ]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]
        
   
    return word_review

In [ ]:
stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
okt = Okt()
clean_train_review = []

for review in train_data['document']:
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_train_review.append([])  #string이 아니면 비어있는 값 추가

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)

word_vocab = tokenizer.word_index # 단어 사전 형태
word_vocab["<PAD>"] = 0

In [ ]:
MAX_SEQUENCE_LENGTH = 8 # 문장 최대 길이

train_inputs = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 학습 데이터를 벡터화
train_labels = np.array(train_data['label']) # 학습 데이터의 라벨

In [ ]:
bluechip_model = Sequential([
    layers.Embedding(vocab_size, 10, input_length=1)
    layers.LSTM(50, return_sequences=True)
    layers.LSTM(50)
    layers.Dropout(0.3)
    layers.Dense(512)
    layers.Dense(output_size, activation='softmax')
]) 
bluchip_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

promising_model = Sequential([
    layers.Embedding(vocab_size, 10, input_length=1)
    layers.LSTM(50, return_sequences=True)
    layers.LSTM(50)
    layers.Dropout(0.3)
    layers.Dense(512)
    layers.Dense(output_size, activation='softmax')
]) 
promising_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

bhistory = bluechip_model.fit(train_inputs, train_labels, batch_size=32, epochs=10)
phistory = promising_model.fit(train_inputs, train_labels, batch_size=32, epochs=10)

In [ ]:
# 환율 가중치
exw = 
# 재무제표 가중치
fw =
# 모델 가중치
w =

In [ ]:
s = input("메뉴선택 (우량주 검색-1, 유망주 검색-2) : ")
if s==0:
    forecast = exw * exchange + fw * financial + w * bluechip_model.predict(inputs)
elif s==1:
    forecast = exw * exchange + fw * financial + w * promising_model.predict(inputs)